# Handling Missing Data

Will Badart <badart_william@bah.com>

created: **SEP 2018**

This notebook is an exploration of different techniques for handling missing data (particularly, large swaths of missing data). We will compare how each technique affects models' performance. I'll be using [this][SO] Stack Overflow post as an outline for the different techniques we'll explore.

[SO]: https://stackoverflow.com/a/35684975/4025659

## The Dataset

I'll be using the [breast cancer][dataset] dataset from `sklearn` as a base, and performing a (reverse?) pre-processing step of removing the values from a random sampling of cells to simulate the problem of missing data.

[dataset]: http://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html#sklearn.datasets.load_breast_cancer

In [1]:
import numpy as np

from random import (
    choices, random, randint, seed as seed_py)
from sklearn.datasets import load_breast_cancer

RANDOM_STATE = 0xdecafbad
PROB_MISSING = 0.9

seed_py(RANDOM_STATE)

def should_i_do_it():
    return random() < PROB_MISSING

def stomp_indexes(x):
    options = list(range(len(x)))
    return choices(options, weights=[6, 3, 1] * 10,
                   k=randint(0, len(x)))

def append_col(arr, newcol):
    assert len(arr) == len(newcol)
    return np.append(
        arr, newcol.reshape(len(newcol), 1), axis=1)

X, y = load_breast_cancer(return_X_y=True)

for x in X:
    if should_i_do_it():
        targets = stomp_indexes(x)
        x[targets] = np.nan

Xy = append_col(X, y)

**NOTE:** The weights here are a cycle of 30 values which alternate between `6`, `3`, and `1`. The acheived effect is that 10 of the features are missing, on average, more than 20% of the time, 10 of the features 15-20% of the time, and the remaining 10 less than 10% of the time.

This creates a distinction between "good" features (with fewer missing values) and bad ones.

## Assessing the Damage

Below, I note a few summary statistics to give an idea of the distribution of missing values. First, we note the proportion of values which have been squashed.

In [2]:
import matplotlib.pyplot as plt

def proportion_na(col):
    return sum(np.isnan(col)) / len(col)

proportions = [proportion_na(col) for col in X.T]
ax = plt.gca()
ax.bar(range(len(proportions)), proportions)
ax.set_ylim([0, 1])
plt.show()

<Figure size 640x480 with 1 Axes>

Please review the above proportions and adjust `PROB_MISSING` and the `weights` keyword argument to `choices` to your preference.

Here, the `count` row shows the number of non-missing values in the column.

In [3]:
import pandas as pd

def describe(A):
    return pd.DataFrame(A).describe()
describe(Xy)

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
count,284.000000,378.000000,490.000000,300.000000,394.000000,500.000000,307.000000,403.000000,497.000000,305.000000,...,292.000000,391.000000,485.000000,294.000000,414.000000,496.000000,290.000000,400.000000,499.000000,569.000000
mean,13.999511,19.362910,92.684816,640.283667,0.096308,0.105046,0.091598,0.049484,0.181258,0.063617,...,25.634623,107.386957,889.244536,0.130969,0.259754,0.271677,0.114128,0.292258,0.084485,0.627417
std,3.548366,4.200937,25.110112,329.493834,0.014474,0.053688,0.079407,0.038586,0.027250,0.007578,...,6.355213,33.923330,586.330346,0.022498,0.160580,0.207228,0.065040,0.064015,0.018390,0.483918
min,6.981000,9.710000,43.790000,178.800000,0.052630,0.019380,0.000000,0.000000,0.116700,0.050240,...,12.490000,50.410000,223.600000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040,0.000000
25%,11.600000,16.497500,75.225000,422.825000,0.086002,0.065402,0.029805,0.020820,0.161900,0.058710,...,21.160000,83.640000,513.100000,0.116600,0.151375,0.115925,0.065510,0.251700,0.072045,0.000000
50%,13.255000,18.885000,86.545000,549.800000,0.096620,0.094035,0.066510,0.033500,0.179200,0.061940,...,25.180000,97.960000,688.600000,0.130100,0.220900,0.226550,0.098560,0.282600,0.080200,1.000000
75%,15.622500,21.870000,106.825000,751.325000,0.104900,0.130600,0.136800,0.074490,0.195600,0.067390,...,29.902500,127.750000,1088.000000,0.143475,0.339200,0.381400,0.160500,0.318925,0.092320,1.000000
max,27.220000,39.280000,188.500000,2010.000000,0.163400,0.345400,0.363500,0.201200,0.304000,0.095750,...,47.160000,229.300000,4254.000000,0.222600,1.058000,1.252000,0.286700,0.663800,0.207500,1.000000


Below is the proportion of data objects with no missing values at all:

In [4]:
len([x for x in X if not any(np.isnan(x))]) / len(X)

0.14411247803163443

## Dealing with it

The five strategies I'm going to try are:

1. *Drop rows with missing data:* it's exactly what it sounds like
2. *Mean/ mode:* fill missing cells with the mean (if we had categorical attributes, mode) of the present values of the column
3. *Conditional mean/mode:* same as (2) but only take the mean of rows which share your label
4. *Hot-decking:* use a distance metric to find the closest row which has a value in your missing column, and use that
5. *KNN:* same as hot-deck but *K* > 1.

### 1. Drop rows with missing values

Since (1) affects the shape of `X`, there also a little extra handling that needs to be done for `y`:

In [5]:
def filter_na(A):
    combined = np.array([
        x for x in A if not any(np.isnan(x))])
    return combined

Xy_dropped = filter_na(Xy)
describe(Xy_dropped)

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
count,82.000000,82.000000,82.000000,82.000000,82.000000,82.000000,82.000000,82.000000,82.000000,82.000000,...,82.000000,82.000000,82.000000,82.000000,82.000000,82.000000,82.000000,82.000000,82.000000,82.000000
mean,14.447646,19.555366,94.414390,679.569512,0.098719,0.116694,0.105048,0.056295,0.187705,0.064219,...,26.212439,111.067317,929.229268,0.135705,0.290301,0.321272,0.128930,0.296994,0.087926,0.585366
std,3.439532,4.812382,23.558927,337.433993,0.015431,0.053048,0.080211,0.038647,0.024726,0.007584,...,6.626186,33.771724,580.762833,0.023949,0.160663,0.217179,0.062592,0.053157,0.020013,0.495691
min,8.219000,10.380000,53.270000,203.900000,0.066130,0.035150,0.001546,0.005495,0.134900,0.052550,...,12.490000,58.080000,249.800000,0.087740,0.054450,0.007732,0.023810,0.178300,0.056950,0.000000
25%,12.077500,16.397500,77.490000,447.775000,0.087505,0.076260,0.041278,0.024890,0.170425,0.059018,...,21.740000,84.207500,524.350000,0.119600,0.171350,0.148025,0.080557,0.265650,0.073472,0.000000
50%,13.855000,18.725000,90.375000,593.250000,0.097455,0.107050,0.083310,0.044355,0.187950,0.062930,...,24.895000,105.200000,736.400000,0.136600,0.251250,0.312900,0.109550,0.290600,0.084930,1.000000
75%,16.697500,22.430000,108.700000,863.575000,0.106575,0.151025,0.165175,0.087213,0.201950,0.067065,...,31.112500,129.625000,1209.250000,0.148175,0.383625,0.460150,0.182650,0.320900,0.099158,1.000000
max,25.730000,39.280000,174.200000,2010.000000,0.139800,0.277600,0.336800,0.191300,0.241900,0.084500,...,44.870000,229.300000,3234.000000,0.222600,0.791700,1.170000,0.275600,0.460100,0.148600,1.000000


The rest of the strategies don't change `y`, but some need to consider it.

### 2. Fill with column mean

In [6]:
def fill_mean(ax):
    ax[np.isnan(ax)] = ax[~np.isnan(ax)].mean()
    return ax

X_mean = np.copy(X)
%timeit np.apply_along_axis(fill_mean, 0, X_mean)
Xy_mean = append_col(X_mean, y)
describe(Xy_mean)

597 µs ± 50.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
count,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,13.999511,19.362910,92.684816,640.283667,0.096308,0.105046,0.091598,0.049484,0.181258,0.063617,...,25.634623,107.386957,889.244536,0.130969,0.259754,0.271677,0.114128,0.292258,0.084485,0.627417
std,2.504652,3.422494,23.298551,239.060993,0.012039,0.050321,0.058283,0.032462,0.025465,0.005544,...,4.548858,28.109726,541.241246,0.016159,0.136928,0.193454,0.046393,0.053653,0.017219,0.483918
min,6.981000,9.710000,43.790000,178.800000,0.052630,0.019380,0.000000,0.000000,0.116700,0.050240,...,12.490000,50.410000,223.600000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040,0.000000
25%,13.270000,17.680000,76.850000,531.500000,0.090560,0.068290,0.058350,0.026390,0.163800,0.061330,...,25.090000,88.840000,531.200000,0.129800,0.175100,0.129500,0.097830,0.266000,0.073430,0.000000
50%,13.999511,19.362910,90.310000,640.283667,0.096308,0.104700,0.091598,0.049484,0.181258,0.063617,...,25.634623,107.386957,760.200000,0.130969,0.259754,0.265400,0.114128,0.292258,0.082550,1.000000
75%,13.999511,20.220000,101.200000,640.283667,0.100300,0.127500,0.091598,0.055980,0.193500,0.063617,...,25.634623,110.600000,932.700000,0.131100,0.279100,0.361700,0.114128,0.302000,0.090750,1.000000
max,27.220000,39.280000,188.500000,2010.000000,0.163400,0.345400,0.363500,0.201200,0.304000,0.095750,...,47.160000,229.300000,4254.000000,0.222600,1.058000,1.252000,0.286700,0.663800,0.207500,1.000000


### 3. Conditional fill with column mean

In [7]:
def fill_cond_mean(A):
    for row in A:
        same_class = A[A[:, -1] == row[-1]]
        for j, v in enumerate(row):
            if np.isnan(v):
                col = same_class[:, j]
                row[j] = col[~np.isnan(col)].mean()
    return A

Xy_cond = np.copy(Xy)
%timeit fill_cond_mean(Xy_cond)
describe(Xy_cond)

29.2 ms ± 869 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
count,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,14.014917,19.370562,92.570461,643.146693,0.096302,0.104952,0.089845,0.048586,0.181228,0.063634,...,25.742782,107.056690,890.530307,0.131475,0.260767,0.273264,0.115233,0.292538,0.084433,0.627417
std,3.110506,3.557774,24.320549,288.552174,0.012412,0.051541,0.069532,0.036142,0.025651,0.005547,...,4.915499,31.843346,566.034653,0.017508,0.144919,0.199742,0.058615,0.055607,0.017345,0.483918
min,6.981000,9.710000,43.790000,178.800000,0.052630,0.019380,0.000000,0.000000,0.116700,0.050240,...,12.490000,50.410000,223.600000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040,0.000000
25%,12.011514,17.680000,76.850000,462.861053,0.090560,0.068290,0.046427,0.025515,0.163800,0.061330,...,23.712011,86.316955,531.200000,0.124132,0.175100,0.129500,0.076086,0.266000,0.073430,0.000000
50%,12.340000,18.320000,85.260000,496.600000,0.094010,0.090090,0.046427,0.029250,0.177900,0.063867,...,23.712011,93.630000,672.400000,0.125600,0.205700,0.221000,0.082350,0.277000,0.079777,1.000000
75%,17.388571,21.539929,109.700000,946.740909,0.103367,0.131800,0.162959,0.083400,0.193500,0.063867,...,29.162524,136.100000,1210.000000,0.143840,0.373500,0.397600,0.181156,0.327069,0.092274,1.000000
max,27.220000,39.280000,188.500000,2010.000000,0.163400,0.345400,0.363500,0.201200,0.304000,0.095750,...,47.160000,229.300000,4254.000000,0.222600,1.058000,1.252000,0.286700,0.663800,0.207500,1.000000


### 5. KNN

In [8]:
from functools import partial
from heapq import nsmallest
from scipy.spatial.distance import euclidean

def skip_by_index(target_idx, a):
    return a[[i for i, _ in enumerate(a) if i != target_idx]]

def euc_with_missing(missing_idx, x, y):
    """
    Compute the euclidean distance between x and y. This will
    get called when x is missing its value at missing_idx. If
    y is also missing this value, it is disqualified. Also, y
    must have a value for all of x's non-missing values.
    """
    x_nan, y_nan = np.isnan(x), np.isnan(y)
    if y_nan[missing_idx] or np.isnan(y[~x_nan]).any():
        return float('inf')
    x_nonan = x[~x_nan]
    return euclidean(x_nonan, y[~x_nan])
    
def knn(A, x, skip, k=5):
    return np.array(nsmallest(
        k, A, key=partial(euc_with_missing, skip, x)))

def fill_knn(A, k=5):
    for row in A:
        for j, v in enumerate(row):
            if np.isnan(v):
                neighbors = knn(A, row, j, k)
                col = neighbors[:, j]
                assert not np.isnan(col).any()
                row[j] = col.mean()
    return A

X_knn = np.copy(X)
%timeit fill_knn(X_knn, int(len(X) ** .5))
Xy_knn = append_col(X_knn, y)
describe(Xy_knn)

19.7 ms ± 514 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
count,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,14.025612,19.295983,91.988199,644.016876,0.096349,0.105056,0.091449,0.049030,0.181327,0.063672,...,25.545723,107.141888,881.061508,0.131842,0.260891,0.271581,0.118323,0.293924,0.084538,0.627417
std,3.303903,3.569145,24.399331,315.838071,0.012316,0.051325,0.067895,0.036089,0.025713,0.005929,...,4.818194,32.500492,567.960024,0.016714,0.144072,0.198343,0.057286,0.055396,0.017347,0.483918
min,6.981000,9.710000,43.790000,178.800000,0.052630,0.019380,0.000000,0.000000,0.116700,0.050240,...,12.490000,50.410000,223.600000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040,0.000000
25%,11.850000,17.150000,75.308654,430.151339,0.089680,0.068150,0.043920,0.022822,0.163800,0.061050,...,22.814865,84.530000,515.900000,0.125282,0.167800,0.127500,0.077837,0.265340,0.073430,0.000000
50%,13.394767,18.831496,86.340000,553.345277,0.096136,0.094620,0.069500,0.036106,0.179800,0.062721,...,25.160000,98.370000,688.170906,0.131427,0.225600,0.226000,0.100431,0.286300,0.081160,1.000000
75%,15.610000,21.410000,104.100000,781.000000,0.102400,0.130300,0.130700,0.073400,0.194500,0.065010,...,27.990000,125.400000,1084.000000,0.139078,0.337100,0.382900,0.162300,0.318700,0.091850,1.000000
max,27.220000,39.280000,188.500000,2010.000000,0.163400,0.345400,0.363500,0.201200,0.304000,0.095750,...,47.160000,229.300000,4254.000000,0.222600,1.058000,1.252000,0.286700,0.663800,0.207500,1.000000


### 4. Hot-decking

In [9]:
def fill_hotdeck(A):
    return fill_knn(A, 1)

X_hotdeck = np.copy(X)
%timeit fill_hotdeck(X_hotdeck)
Xy_hotdeck = append_col(X_hotdeck, y)
describe(Xy_hotdeck)

18.8 ms ± 201 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
count,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,14.111248,19.253427,92.007258,651.688049,0.096113,0.105008,0.094815,0.049947,0.180953,0.063892,...,25.630562,107.627926,880.053779,0.131940,0.263855,0.275683,0.118295,0.295298,0.084555,0.627417
std,3.500177,4.031181,24.440303,342.707112,0.014319,0.053108,0.080330,0.038644,0.027393,0.007427,...,6.217015,34.027234,572.366937,0.022391,0.164256,0.215272,0.066487,0.064510,0.018373,0.483918
min,6.981000,9.710000,43.790000,178.800000,0.052630,0.019380,0.000000,0.000000,0.116700,0.050240,...,12.490000,50.410000,223.600000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040,0.000000
25%,11.710000,16.490000,74.720000,417.200000,0.085880,0.066010,0.033280,0.020690,0.161900,0.058880,...,21.330000,83.670000,510.500000,0.117800,0.151600,0.114400,0.068760,0.255100,0.071850,0.000000
50%,13.270000,18.660000,86.490000,542.900000,0.095790,0.094620,0.067370,0.036130,0.179200,0.062290,...,25.220000,97.820000,687.600000,0.131200,0.220800,0.226700,0.099100,0.284900,0.080200,1.000000
75%,15.660000,21.810000,104.100000,782.600000,0.104600,0.131000,0.147800,0.077850,0.195400,0.067140,...,30.500000,126.900000,1050.000000,0.144400,0.341600,0.386500,0.170800,0.321600,0.092510,1.000000
max,27.220000,39.280000,188.500000,2010.000000,0.163400,0.345400,0.363500,0.201200,0.304000,0.095750,...,47.160000,229.300000,4254.000000,0.222600,1.058000,1.252000,0.286700,0.663800,0.207500,1.000000


## Showdown

So which enriched dataset yields the best model? Let's find out.

In [10]:
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

MAX_DEPTH = 12

def split_labels(Xy):
    return Xy[:, :-1], Xy[:, -1]

data = {
    'dropped': split_labels(Xy_dropped),
    'mean': split_labels(Xy_mean),
    'cond': split_labels(Xy_cond),
    'knn': split_labels(Xy_knn),
    'hotdeck': split_labels(Xy_hotdeck) }

for name, (X, y) in data.items():
    print(name)
    model = DecisionTreeClassifier(max_depth=MAX_DEPTH, random_state=RANDOM_STATE)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=RANDOM_STATE)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print('accuracy:', accuracy_score(y_test, y_pred))
    print('f1-score:', f1_score(y_test, y_pred))
    print()

dropped
accuracy: 0.9047619047619048
f1-score: 0.9230769230769231

mean
accuracy: 0.9020979020979021
f1-score: 0.9230769230769231

cond
accuracy: 0.972027972027972
f1-score: 0.9784946236559139

knn
accuracy: 0.9370629370629371
f1-score: 0.9518716577540107

hotdeck
accuracy: 0.9440559440559441
f1-score: 0.956989247311828

